# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


In [1]:
# # 필요한 라이브러리를 설치합니다.
# !pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [4]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        #pbar.set_description(f"Loss: {loss.item():.4f}") # loss 보여주는거 없앴어 시간 오래걸려서

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

In [5]:
# one epoch 학습을 위한 함수입니다.
def val_one_epoch(loader, model, optimizer, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)
        with torch.no_grad():
            preds = model(image)
            loss = loss_fn(preds, targets)
        val_loss += loss.item() 
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": train_acc,
        "val_f1": train_f1,
    }

    return ret, train_f1

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# data config
data_path = 'data/'

# model config
model_name = 'resnet50' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 0

# early stop count 
stop_count = 7

cuda


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

import torchvision.transforms as tf
preprocessing_pipeline = transforms.Compose([
    tf.RandomCrop(size=(200,200)),
    tf.ColorJitter(brightness=0.5, contrast= 0.5, saturation=0.5,hue=0.5),
    tf.RandomRotation(degree=45),
    tf.GaussianBlur(kernel_size=5),
    tf.ToTensor(),
    tf.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

# val_dataset 생성

In [7]:
import os
import shutil
import random

# 데이터 폴더와 타겟 폴더 설정
data_folder = './data/rotated_train'
target_folders = [str(num) for num in range(17)]  # 각 타겟 폴더의 이름

# 테스트 데이터 비율 설정
test_ratio = 0.2 # 0.3, 0.2, 0.4

# 각 타겟 폴더에서 테스트 데이터를 저장할 폴더 설정
train_folder = './data/split_train_data'
test_folder = './data/split_val_data'
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Train 데이터 프레임 생성
train_data = []
# Test 데이터 프레임 생성
test_data = []

def make_val_dataset(target_folders, data_folder, train_folder, test_folder):
    # 각 타겟 폴더 별로 데이터를 분리하여 저장
    for target in target_folders:
        target_path = os.path.join(data_folder, target)
        train_target_path = os.path.join(train_folder)
        test_target_path = os.path.join(test_folder)

        # 타겟 폴더의 이미지 목록 가져오기
        images = os.listdir(target_path)
        # 테스트 데이터의 개수 계산
        num_test_images = int(len(images) * test_ratio)
        # 이미지를 랜덤하게 선택하여 테스트 데이터와 트레인 데이터로 나누기
        test_images = random.sample(images, num_test_images)
        train_images = [image for image in images if image not in test_images]

        # 테스트 데이터 이동
        for image in test_images:
            #src = os.path.join(target_path, image)
            #dst = os.path.join(test_target_path, image)
            test_data.append([image, target]) # image랑 target 쉼표 기준으로 csv 파일 만듦
            img_path = os.path.join(target_path, image)
            real_image = Image.open(img_path)
            real_image.save(test_target_path + '/' + image)
            #shutil.move(src, dst)

        # 트레인 데이터 이동
        for image in train_images:
            #src = os.path.join(target_path, image)
            #dst = os.path.join(train_target_path, image)
            train_data.append([image, target]) # image랑 target 쉼표 기준으로 csv 파일 만듦
            img_path = os.path.join(target_path, image)
            real_image = Image.open(img_path)
            real_image.save(train_target_path + '/' + image)
            #shutil.move(src, dst)

    # Train 데이터 CSV 파일로 저장
    train_df = pd.DataFrame(train_data, columns=['ID', 'target'])
    train_df.to_csv('./data/split_train.csv', index=False)
    # Test 데이터 CSV 파일로 저장
    test_df = pd.DataFrame(test_data, columns=['ID', 'target'])
    test_df.to_csv('./data/split_val.csv', index=False)
    return 0

make_val_dataset(target_folders, data_folder, train_folder, test_folder)

0

# transform 코드 생성하기 

In [8]:
# #augmentation을 위한 transform 코드
# trn_transform = A.Compose([
#     # 이미지 크기 조정
#     A.Resize(height=img_size, width=img_size),
#     # images normalization
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
#     ToTensorV2(),
# ])
import cv2
#import albumentations as A
# p 함수가 적용될 확률
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    # 좌우 반전
    A.HorizontalFlip(p=0.5), 
    # 상하 반전
    A.VerticalFlip(p=0.5), 
    # 수평,수직 또는 수평 및 수직으로 뒤집는다.
    A.Flip(always_apply=False, p=0.5),
    
    # 행과 열의 입력을 바꿈
    A.Transpose(always_apply=False, p=0.5),
    
    #  -90 ~ 90도 사이로 돌림, 남은 공간은 주변 환경으로 채움 (cv2.BORDER_CONSTAN, cv2.BORDER_REFLECT, cv2.BORDER_REFLECT_101, cv2.BORDER_WRAP)
    A.Rotate(limit=5, p=0.3, border_mode=cv2.BORDER_REPLICATE), 
        
    # shift, scale, rotate를 한번에 적용
    A.ShiftScaleRotate(shift_limit=0.4, scale_limit=(0.5, 0.9), rotate_limit=90, p=0.5, border_mode=cv2.BORDER_REPLICATE),
    # scale만 적용한 경우 (이미지 확대)
    A.ShiftScaleRotate(shift_limit=0, scale_limit=(0.5, 0.9), rotate_limit=0, p=0.5),
    # shift만 적용한 경우 (height, width)
    A.ShiftScaleRotate(shift_limit=(0.4, 0.5), scale_limit=0, rotate_limit=0, p=0.5,  border_mode=cv2.BORDER_REPLICATE),
    # Crop은 원본 이미지의 특정 영역을 잘라낸 후, 잘라낸 사이즈를 반환
    #A.Crop(x_min=int(img_size/4), y_min=int(img_size/4), x_max=int(img_size/2), y_max=int(img_size/2), p=0.3),
    # 이미지의 중심을 기준으로 입력된 height, width값 만큼의 영역을 잘라낸 후, 잘라낸 사이즈를 반환(반드시 Resize 해주어야함)
    #A.CenterCrop(width=img_size, height=img_size, p=1),
    #A.Resize(img_size, img_size),
    
    # 밝기와 대비 변경 (대비를 올리면 어두운색은 더 어둡게, 밝은색은 더 밝게)
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=0.5),
    # 밝기만 변경
    A.RandomBrightnessContrast(brightness_limit=(-0.8, 0.8), contrast_limit=0, p=0.5),
    # 대비만 변경
    A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(-0.8, 0.8), p=0.5),
    # 색상 채도 명도 변경, default(hue_shift_limit=(-20, 20), sat_shift_limit=(-30, 30), val_shift_limit=(-20, 20))
    A.HueSaturationValue(p=0.3),
    
    #RGB 값 각각 범위내 임의로 변경 default(r_shift_limit=(-20, 20), g_shift_limit=(-20, 20), b_shift_limit=(-20, 20))
    A.RGBShift(p=0.3),
    
    #RGB Channel을 랜덤하게 섞음
    A.ChannelShuffle(p=0.3),
    
    #가우시안 노이즈 분포를 가지는 노이즈를 추가
    A.GaussNoise(p=0.5, var_limit=(100, 200)),
    # 가우시안 필터 적용
    A.GaussianBlur( blur_limit = 7 , always_apply = False , p = 0.5 ),
    # 글래스 필터 사용
    A.GlassBlur( sigma = 0.7 , max_delta = 4 , iterations = 2 , always_apply = False , mode = 'fast' , p = 0.5 ),
    #정사각형 노이즈 추가
    A.Cutout(p=0.3, num_holes=8, max_h_size=24, max_w_size=24),
    
    # blur_limit가 클수록 더 흐림
    A.Blur(p=0.5, blur_limit=(50, 60)),
    
    # Random으로 이미지 resize 후 crop
    A.RandomResizedCrop(height=img_size, width=img_size, p=0.5),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

# val image 변환을 위한 transform 코드
val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:184: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [9]:
# # Dataset 정의
trn_dataset = ImageDataset(
    "data/split_train.csv",
    "data/split_train_data/",
    transform=trn_transform
)

# val _ data만듬
val_dataset = ImageDataset(
    "data/split_val.csv",
    "data/split_val_data/",
    transform = val_transform
)

# 기존 3500장짜리 서버제출할때 넣는 데이터 
tst_dataset = ImageDataset(
    "data/sample_submission.csv",
    "data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(val_dataset), len(tst_dataset))

183377 45843 3140


In [10]:
display('trn_dataset', trn_dataset[0][0].size())
display('val_dataset', val_dataset[0][0].size())
display('tst_dataset', tst_dataset[0][0].size())

'trn_dataset'

torch.Size([3, 256, 256])

'val_dataset'

torch.Size([3, 256, 256])

'tst_dataset'

torch.Size([3, 256, 256])

In [11]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [12]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [13]:
import pickle

# ---------- 변수 건들면 안돼
best_f1_score = 0
stop_early_count = 0

# EPOCHS
for epoch in range(EPOCHS): 
   if stop_early_count == stop_count:
      break
   ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
   ret2, _val_f1 = val_one_epoch(val_loader, model, optimizer, loss_fn, device=device)
        
   print(f"Epoch: {epoch}, Loss: {ret['train_loss']:.4f}, Accuracy: {ret['train_acc']:.4f}, F1-Score: {ret['train_f1']:.4f}")
   print(f"validation Loss: {ret['val_loss']:.4f}, val_Accuracy: {ret2['val_acc']:.4f}, val_F1-Score: {ret2['val_f1']:.4f}")
    
   if _val_f1 > best_f1_score:
      best_f1_score = _val_f1
      stop_early_count = 0
      # 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
      with open('saved_model.pkl', 'wb') as f:
        pickle.dump(model, f)
   
   else:
      stop_early_count += 1

print(best_f1_score)

100%|██████████| 1433/1433 [03:04<00:00,  7.78it/s]


Epoch: 0, Loss: 2.2739, Accuracy: 0.2470, F1-Score: 0.2231
val, val_Accuracy: 0.6901, val_F1-Score: 0.6692


100%|██████████| 1433/1433 [03:04<00:00,  7.75it/s]


Epoch: 1, Loss: 2.0668, Accuracy: 0.3140, F1-Score: 0.2906
val, val_Accuracy: 0.7033, val_F1-Score: 0.6902


100%|██████████| 1433/1433 [03:04<00:00,  7.77it/s]


...

100%|██████████| 1433/1433 [03:08<00:00,  7.62it/s]


Epoch: 25, Loss: 1.6141, Accuracy: 0.4623, F1-Score: 0.4469
val, val_Accuracy: 0.9014, val_F1-Score: 0.8987


100%|██████████| 1433/1433 [03:05<00:00,  7.72it/s]


Epoch: 26, Loss: 1.6075, Accuracy: 0.4632, F1-Score: 0.4483
val, val_Accuracy: 0.8345, val_F1-Score: 0.8287


 34%|███▍      | 1977/5731 [17:43<34:36,  1.81it/s]

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
import pickle
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)
    
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:14<00:00,  6.66it/s]


In [ ]:
model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

In [ ]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [ ]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("pred.csv", index=False)

In [ ]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,16
2,00396fbc1f6cc21d.jpg,8
3,00471f8038d9c4b6.jpg,8
4,00901f504008d884.jpg,2
